In [2]:
import pandas as pd

# Ruta del archivo CSV
ruta_csv = '/Users/macmontxinho/Desktop/Teams/Wyscout/data/Jugadores.csv'

# Cargar datos
df = pd.read_csv(ruta_csv, sep=';')

# Filtrar solo delanteros en Primera RFEF con al menos 1000 minutos
df_delanteros = df[
    (df['League'] == 'Primera RFEF') &
    (df['position_gen_ESP1'] == 'Delantero') &
    (df['minutes_on_field'] >= 1000)
].copy()

# Diccionario con las métricas y sus pesos por categoría
metricas_pesos = {
    # 1. Generación de ocasiones
    'key_passes_avg': 0.06,
    'pre_assist_avg': 0.06,
    'smart_passes_avg': 0.06,
    'pass_to_penalty_area_avg': 0.05,
    'accurate_pass_to_penalty_area_percent': 0.05,
    'shot_assists_avg': 0.05,
    'touch_in_box_avg': 0.07,

    # 2. Desborde y progresión
    'dribbles_avg': 0.05,
    'successful_dribbles_percent': 0.05,
    'progressive_run_avg': 0.05,
    'progressive_pass_avg': 0.05,
    'successful_progressive_pass_percent': 0.05,

    # 3. Aporte ofensivo directo
    'xg_shot_avg': 0.08,
    'shots_avg': 0.05,
    'shots_on_target_percent': 0.05,
    'non_penalty_goal_avg': 0.05,
    'assists_avg': 0.05,
    'xg_assist_avg': 0.05,

    # 4. Eficiencia con pocos minutos (penaliza muchos minutos)
    'minutes_on_field': 0.01  # peso negativo aplicado luego
}

# Calcular rankings normalizados para cada métrica
for col, peso in metricas_pesos.items():
    if col == 'minutes_on_field':
        df_delanteros[f'{col}_rank'] = df_delanteros[col].rank(pct=True, ascending=True)  # menos minutos, mejor
    else:
        df_delanteros[f'{col}_rank'] = df_delanteros[col].rank(pct=True)

# Índice ponderado de impacto ofensivo
df_delanteros['impacto_ofensivo'] = sum(
    df_delanteros[f'{col}_rank'] * peso for col, peso in metricas_pesos.items()
)

# Mostrar el top 10
columnas_mostrar = ['full_name', 'current_team_name', 'minutes_on_field', 'goals', 'assists_avg', 'xg_shot_avg', 'impacto_ofensivo']
top_10 = df_delanteros.sort_values('impacto_ofensivo', ascending=False).head(10)
print(top_10[columnas_mostrar])

                           full_name    current_team_name  minutes_on_field  \
97              Antón Escobar Tapias     Cultural Leonesa              1743   
913          Víctor Muñoz Villanueva          Real Madrid              2928   
920            Gonzalo García Torres          Real Madrid              3425   
133           Antonio Cortes Heredia  Gimnàstic Tarragona              3130   
121           Lautaro de León Billar           FC Andorra              2971   
283          Adrián Fuentes González             Tarazona              3074   
7          José María Callejón Bueno             Marbella              3617   
657          Maroan Sannadi Harrouch      Athletic Bilbao              1878   
755  Luis Eduardo Quintero Hernández         Villarreal B              1428   
305             Daniel Aquino Pintos                Ceuta              1725   

     goals  assists_avg  xg_shot_avg  impacto_ofensivo  
97      12         0.21         0.61          0.832121  
913     11      

In [ ]:
import pandas as pd

# Ruta al archivo
ruta_csv = '/Users/macmontxinho/Desktop/Teams/Wyscout/data/Jugadores.csv'

# Cargar el CSV (ajusta sep=';' si el delimitador es punto y coma)
df = pd.read_csv(ruta_csv, sep=';')

# Lista de clubes válidos
clubes_validos = [
    'intercity sj d\' alacant', 'ibiza', 'marbella', 'gimnàstic tarragona', 'fc andorra',
    'lugo', 'real murcia', 'zamora', 'unionistas de salamanca',
    'cultural leonesa', 'alcorcón', 'mérida ad', 'arenteiro', 'alcoyano', 'antequera',
    'ceuta', 'fuenlabrada', 'ponferradina', 'ourense cf', 'sestao river', 'real unión',
    'algeciras', 'hércules', 'recreativo huelva', 'tarazona', 'atlético sanluqueño',
    'yeclano', 'amorebieta', 'barakaldo', 'gimnástica segoviana'
]

# Filtrar jugadores: Primera RFEF, Delantero, >=1000 minutos, y en clubes seleccionados
df_delanteros = df[
    (df['League'].str.lower() == 'primera rfef') &
    (df['position_gen_ESP1'].str.lower() == 'delantero') &
    (df['minutes_on_field'] >= 1000) &
    (df['last_club_name'].str.lower().isin(clubes_validos))
].copy()

# Diccionario con métricas y pesos
metricas_pesos = {
    # Generación de ocasiones
    'key_passes_avg': 0.06,
    'pre_assist_avg': 0.06,
    'smart_passes_avg': 0.06,
    'pass_to_penalty_area_avg': 0.05,
    'accurate_pass_to_penalty_area_percent': 0.05,
    'shot_assists_avg': 0.05,
    'touch_in_box_avg': 0.07,

    # Desborde y progresión
    'dribbles_avg': 0.05,
    'successful_dribbles_percent': 0.05,
    'progressive_run_avg': 0.05,
    'progressive_pass_avg': 0.05,
    'successful_progressive_pass_percent': 0.05,

    # Aporte ofensivo directo
    'xg_shot_avg': 0.08,
    'shots_avg': 0.05,
    'shots_on_target_percent': 0.05,
    'non_penalty_goal_avg': 0.05,
    'assists_avg': 0.05,
    'xg_assist_avg': 0.05,

    # Eficiencia con pocos minutos
    'minutes_on_field': 0.01  # penaliza muchos minutos
}

# Calcular percentiles
for col, peso in metricas_pesos.items():
    ascending = True if col == 'minutes_on_field' else False
    df_delanteros[f'{col}_rank'] = df_delanteros[col].rank(pct=True, ascending=ascending)

# Calcular índice ponderado
df_delanteros['impacto_ofensivo'] = sum(
    df_delanteros[f'{col}_rank'] * peso for col, peso in metricas_pesos.items()
)

# Mostrar top 25
columnas_mostrar = ['full_name', 'last_club_name', 'minutes_on_field', 'goals', 'assists_avg', 'xg_shot_avg', 'impacto_ofensivo']
top_25 = df_delanteros.sort_values('impacto_ofensivo', ascending=False).head(25)

print(top_25[columnas_mostrar])

                       full_name           last_club_name  minutes_on_field  \
1102       Daniel Selma Mercader               amorebieta              2171   
469           Jon Etxaniz Llamas             sestao river              1890   
1101          Álvaro Marín Sesma               amorebieta              1402   
1011        Pau Ferrer Besalduch             ponferradina              1418   
973         Sergi Armero Montané        recreativo huelva              1018   
439      Ilies Hassaine Daouadji              fuenlabrada              1220   
336               Agustín Coscia                 hércules              2112   
770     Javier Carbonell Piserra               ourense cf              2599   
28        Enrique González Casín                    ibiza              1446   
85         Daniel García Beltrán  unionistas de salamanca              1113   
1092        Alzola Aimar Vicandi               amorebieta              1223   
1095           Martín Ochoa Ruiz                    

In [6]:
import pandas as pd

# Ruta al archivo
ruta_csv = '/Users/macmontxinho/Desktop/Teams/Wyscout/data/Jugadores.csv'

# Cargar CSV (ajusta sep=';' si aplica)
df = pd.read_csv(ruta_csv, sep=';')

# Nueva lista de filiales
clubes_filiales = [
    'atlético madrid b',
    'barcelona atlètic',
    'villarreal b',
    'real madrid castilla',
    'osasuna promesas',
    'celta fortuna',
    'sevilla atlético',
    'betis deportivo',
    'bilbao athletic',
    'celta de vigo b',
    'osasuna b',
    'real sociedad b'
]

# Filtro: Primera RFEF, delanteros, mínimo 1000 minutos, y clubes filiales
df_delanteros = df[
    (df['League'].str.lower() == 'primera rfef') &
    (df['position_gen_ESP1'].str.lower() == 'delantero') &
    (df['minutes_on_field'] >= 1000) &
    (df['last_club_name'].str.lower().isin(clubes_filiales))
].copy()

# Diccionario de métricas y pesos
metricas_pesos = {
    # Generación de ocasiones
    'key_passes_avg': 0.06,
    'pre_assist_avg': 0.06,
    'smart_passes_avg': 0.06,
    'pass_to_penalty_area_avg': 0.05,
    'accurate_pass_to_penalty_area_percent': 0.05,
    'shot_assists_avg': 0.05,
    'touch_in_box_avg': 0.07,

    # Desborde y progresión
    'dribbles_avg': 0.05,
    'successful_dribbles_percent': 0.05,
    'progressive_run_avg': 0.05,
    'progressive_pass_avg': 0.05,
    'successful_progressive_pass_percent': 0.05,

    # Aporte ofensivo directo
    'xg_shot_avg': 0.08,
    'shots_avg': 0.05,
    'shots_on_target_percent': 0.05,
    'non_penalty_goal_avg': 0.05,
    'assists_avg': 0.05,
    'xg_assist_avg': 0.05,

    # Eficiencia con pocos minutos
    'minutes_on_field': 0.01  # penaliza muchos minutos
}

# Calcular rankings por percentil
for col, peso in metricas_pesos.items():
    ascending = True if col == 'minutes_on_field' else False
    df_delanteros[f'{col}_rank'] = df_delanteros[col].rank(pct=True, ascending=ascending)

# Índice final
df_delanteros['impacto_ofensivo'] = sum(
    df_delanteros[f'{col}_rank'] * peso for col, peso in metricas_pesos.items()
)

# Mostrar resultados
columnas_mostrar = ['full_name', 'last_club_name', 'minutes_on_field', 'goals', 'assists_avg', 'xg_shot_avg', 'impacto_ofensivo']
top_25_filial = df_delanteros.sort_values('impacto_ofensivo', ascending=False).head(25)

print(top_25_filial[columnas_mostrar])

                              full_name        last_club_name  \
1125          Ekain  Orobengoa Arbelaiz       real sociedad b   
934             Destiny Ilahude Enebeli       betis deportivo   
916                Lorenzo Zúñiga Owono  real madrid castilla   
759             Borislav Ivanov Stankov          villarreal b   
1037           Marcos Fernández Sánchez       betis deportivo   
783                  Asier Hierro Campo       bilbao athletic   
853                   Omar Janneh Susso     atlético madrid b   
836             Carlos Lumbreras Solano      osasuna promesas   
740        Alvaro Miguel Garcia-Pascual      sevilla atlético   
825                        Oscar Marcos         celta fortuna   
1024  Arkaitz Mariezkurrena Etxezarreta       real sociedad b   
717           Iván Cédric Bikoue Embolo     barcelona atlètic   
858                 Adrián Niño Heredia     atlético madrid b   
1056            Leandro Antonetti López      sevilla atlético   
833             Nzubechi 

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# Cargar CSV
ruta_csv = '/Users/macmontxinho/Desktop/Teams/Wyscout/data/Jugadores.csv'
df = pd.read_csv(ruta_csv, sep=';')

# Clubes no filiales
clubes_validos = [
    'intercity sj d\' alacant', 'ibiza', 'marbella', 'gimnàstic tarragona', 'fc andorra',
    'lugo', 'real murcia', 'zamora', 'unionistas de salamanca',
    'cultural leonesa', 'alcorcón', 'mérida ad', 'arenteiro', 'alcoyano', 'antequera',
    'ceuta', 'fuenlabrada', 'ponferradina', 'ourense cf', 'sestao river', 'real unión',
    'algeciras', 'hércules', 'recreativo huelva', 'tarazona', 'atlético sanluqueño',
    'yeclano', 'amorebieta', 'barakaldo', 'gimnástica segoviana'
]

# Clubes filiales
clubes_filiales = [
    'atlético madrid b', 'barcelona atlètic', 'villarreal b', 'real madrid castilla',
    'osasuna promesas', 'celta fortuna', 'sevilla atlético', 'betis deportivo',
    'bilbao athletic', 'celta de vigo b', 'osasuna b', 'real sociedad b'
]

# Métricas y pesos
metricas_pesos = {
    'key_passes_avg': 0.06, 'pre_assist_avg': 0.06, 'smart_passes_avg': 0.06,
    'pass_to_penalty_area_avg': 0.05, 'accurate_pass_to_penalty_area_percent': 0.05,
    'shot_assists_avg': 0.05, 'touch_in_box_avg': 0.07,
    'dribbles_avg': 0.05, 'successful_dribbles_percent': 0.05,
    'progressive_run_avg': 0.05, 'progressive_pass_avg': 0.05,
    'successful_progressive_pass_percent': 0.05,
    'xg_shot_avg': 0.08, 'shots_avg': 0.05, 'shots_on_target_percent': 0.05,
    'non_penalty_goal_avg': 0.05, 'assists_avg': 0.05, 'xg_assist_avg': 0.05,
    'minutes_on_field': 0.01
}

# Función para generar tabla y devolver df_top
def calcular_ranking(df, clubes):
    subset = df[
        (df['League'].str.lower() == 'primera rfef') &
        (df['position_gen_ESP1'].str.lower() == 'delantero') &
        (df['minutes_on_field'] >= 1000) &
        (df['last_club_name'].str.lower().isin(clubes))
    ].copy()

    for col, peso in metricas_pesos.items():
        ascending = True if col == 'minutes_on_field' else False
        subset[f'{col}_rank'] = subset[col].rank(pct=True, ascending=ascending)

    subset['impacto_ofensivo'] = sum(
        subset[f'{col}_rank'] * peso for col, peso in metricas_pesos.items()
    )

    columnas_mostrar = ['full_name', 'last_club_name', 'age', 'minutes_on_field', 'impacto_ofensivo']
    return subset.sort_values('impacto_ofensivo', ascending=False).head(25)[columnas_mostrar]

# Calcular rankings
top_25 = calcular_ranking(df, [c.lower() for c in clubes_validos])
top_25_filial = calcular_ranking(df, [c.lower() for c in clubes_filiales])

# Renombrar columnas para presentación
def renombrar_columnas(df_tabla):
    return df_tabla.rename(columns={
        'full_name': 'Jugador',
        'last_club_name': 'Equipo',
        'age': 'Edad',
        'minutes_on_field': 'Minutos',
        'impacto_ofensivo': 'Índice'
    })

top_25 = renombrar_columnas(top_25)
top_25_filial = renombrar_columnas(top_25_filial)

# 📄 Crear PDF con ambas tablas
with PdfPages("Ranking_Delanteros_Primera_RFEF.pdf") as pdf:
    for df_tabla, titulo in zip(
        [top_25, top_25_filial],
        ["Top 25 Delanteros – Equipos No Filiales", "Top 25 Delanteros – Equipos Filiales"]
    ):
        fig, ax = plt.subplots(figsize=(12, len(df_tabla)*0.5))
        ax.axis('off')
        tabla = ax.table(
            cellText=df_tabla.values,
            colLabels=df_tabla.columns,
            cellLoc='center',
            loc='center'
        )
        tabla.auto_set_font_size(False)
        tabla.set_fontsize(10)
        tabla.scale(1, 1.5)
        plt.title(titulo, fontsize=14, weight='bold')
        pdf.savefig(fig, bbox_inches='tight')
        plt.close()

print("✅ PDF generado: Ranking_Delanteros_Primera_RFEF.pdf")

✅ PDF generado: Ranking_Delanteros_Primera_RFEF.pdf


In [19]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# Cargar CSV
ruta_csv = '/Users/macmontxinho/Desktop/Teams/Wyscout/data/Jugadores.csv'
df = pd.read_csv(ruta_csv, sep=';')

# Clubes no filiales
clubes_validos = [
    'intercity sj d\' alacant', 'ibiza', 'marbella', 'gimnàstic tarragona', 'fc andorra',
    'lugo', 'real murcia', 'zamora', 'unionistas de salamanca',
    'cultural leonesa', 'alcorcón', 'mérida ad', 'arenteiro', 'alcoyano', 'antequera',
    'ceuta', 'fuenlabrada', 'ponferradina', 'ourense cf', 'sestao river', 'real unión',
    'algeciras', 'hércules', 'recreativo huelva', 'tarazona', 'atlético sanluqueño',
    'yeclano', 'amorebieta', 'barakaldo', 'gimnástica segoviana'
]

# Clubes filiales
clubes_filiales = [
    'atlético madrid b', 'barcelona atlètic', 'villarreal b', 'real madrid castilla',
    'osasuna promesas', 'celta fortuna', 'sevilla atlético', 'betis deportivo',
    'bilbao athletic', 'celta de vigo b', 'osasuna b', 'real sociedad b'
]

# Métricas y pesos
metricas_pesos = {
    'key_passes_avg': 0.06, 'pre_assist_avg': 0.06, 'smart_passes_avg': 0.06,
    'pass_to_penalty_area_avg': 0.05, 'accurate_pass_to_penalty_area_percent': 0.05,
    'shot_assists_avg': 0.05, 'touch_in_box_avg': 0.07,
    'dribbles_avg': 0.05, 'successful_dribbles_percent': 0.05,
    'progressive_run_avg': 0.05, 'progressive_pass_avg': 0.05,
    'successful_progressive_pass_percent': 0.05,
    'xg_shot_avg': 0.08, 'shots_avg': 0.05, 'shots_on_target_percent': 0.05,
    'non_penalty_goal_avg': 0.05, 'assists_avg': 0.05, 'xg_assist_avg': 0.05,
    'minutes_on_field': 0.01
}

# Función para calcular ranking
def calcular_ranking(df, clubes):
    subset = df[
        (df['League'].str.lower() == 'primera rfef') &
        (df['position_gen_ESP1'].str.lower() == 'delantero') &
        (df['minutes_on_field'] >= 1000) &
        (df['last_club_name'].str.lower().isin(clubes))
    ].copy()

    for col, peso in metricas_pesos.items():
        ascending = True if col == 'minutes_on_field' else False
        subset[f'{col}_rank'] = subset[col].rank(pct=True, ascending=ascending)

    subset['impacto_ofensivo'] = sum(
        subset[f'{col}_rank'] * peso for col, peso in metricas_pesos.items()
    )
    subset['impacto_ofensivo'] = subset['impacto_ofensivo'].round(3)
    return subset.sort_values('impacto_ofensivo', ascending=False).head(25)

# Ranking
top_25 = calcular_ranking(df, [c.lower() for c in clubes_validos])
top_25_filial = calcular_ranking(df, [c.lower() for c in clubes_filiales])

# Tabla sin imágenes
def crear_tabla(df_tabla, titulo, pdf):
    fig, ax = plt.subplots(figsize=(14, len(df_tabla)*0.6))
    ax.axis('off')

    col_labels = ['Jugador (Equipo)', 'Edad', 'Minutos', 'Índice']
    table_data = []

    for _, row in df_tabla.iterrows():
        jugador_equipo = f"{row['full_name']} ({row['last_club_name']})"
        table_data.append([
            jugador_equipo,
            row['age'],
            int(row['minutes_on_field']),
            round(row['impacto_ofensivo'], 3)
        ])

    tabla = ax.table(
        cellText=table_data,
        colLabels=col_labels,
        cellLoc='center',
        loc='center'
    )
    tabla.auto_set_font_size(False)
    tabla.set_fontsize(10)
    tabla.scale(1.2, 1.3)
    plt.title(titulo, fontsize=16, weight='bold')
    pdf.savefig(fig, bbox_inches='tight')
    plt.close()

# Página de explicación de métricas
def agregar_metrica_explica(pdf):
    texto = """
1. Participación en la generación de ocasiones:
• key_passes_avg, pre_assist_avg, smart_passes_avg
• pass_to_penalty_area_avg, accurate_pass_to_penalty_area_percent
• shot_assists_avg, touch_in_box_avg

2. Capacidad de desborde y progresión:
• dribbles_avg, successful_dribbles_percent
• progressive_run_avg, progressive_pass_avg
• successful_progressive_pass_percent

3. Aporte ofensivo directo (sin depender solo de goles):
• xg_shot_avg, shots_avg, shots_on_target_percent
• non_penalty_goal_avg, assists_avg, xg_assist_avg

4. Eficiencia e impacto con pocos minutos:
• minutes_on_field, goals_avg (o por 90’), xg_shot_avg (o por 90’)
• assists_avg (o por 90’), pre_assist_avg (o por 90’)
    """
    fig, ax = plt.subplots(figsize=(12, 8))
    ax.axis('off')
    ax.text(0, 1, "📊 Métricas utilizadas para el índice de impacto ofensivo", fontsize=16, weight='bold', va='top')
    ax.text(0, 0.95, texto.strip(), fontsize=12, va='top')
    pdf.savefig(fig, bbox_inches='tight')
    plt.close()

# Exportar PDF
with PdfPages("Ranking_Delanteros_Sin_Imagenes.pdf") as pdf:
    crear_tabla(top_25, "Top 25 Delanteros – Equipos No Filiales", pdf)
    crear_tabla(top_25_filial, "Top 25 Delanteros – Equipos Filiales", pdf)
    agregar_metrica_explica(pdf)

print("✅ PDF generado: Ranking_Delanteros_Sin_Imagenes.pdf")

✅ PDF generado: Ranking_Delanteros_Sin_Imagenes.pdf


/var/folders/fl/2m17g4gs6l70lh48xp8n6k880000gn/T/ipykernel_22690/679722840.py:117: UserWarning: Glyph 128202 (\N{BAR CHART}) missing from font(s) DejaVu Sans.
  pdf.savefig(fig, bbox_inches='tight')


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import pandas as pd
import requests
from PIL import Image
from io import BytesIO
from fpdf import FPDF
import os
import tempfile

# Cargar CSV
ruta_csv = '/Users/macmontxinho/Desktop/Teams/Wyscout/data/Jugadores.csv'
df = pd.read_csv(ruta_csv, sep=';')

# Clubes no filiales
clubes_validos = [
    'intercity sj d\' alacant', 'ibiza', 'marbella', 'gimnàstic tarragona', 'fc andorra',
    'lugo', 'real murcia', 'zamora', 'unionistas de salamanca',
    'cultural leonesa', 'alcorcón', 'mérida ad', 'arenteiro', 'alcoyano', 'antequera',
    'ceuta', 'fuenlabrada', 'ponferradina', 'ourense cf', 'sestao river', 'real unión',
    'algeciras', 'hércules', 'recreativo huelva', 'tarazona', 'atlético sanluqueño',
    'yeclano', 'amorebieta', 'barakaldo', 'gimnástica segoviana'
]

# Clubes filiales
clubes_filiales = [
    'atlético madrid b', 'barcelona atlètic', 'villarreal b', 'real madrid castilla',
    'osasuna promesas', 'celta fortuna', 'sevilla atlético', 'betis deportivo',
    'bilbao athletic', 'celta de vigo b', 'osasuna b', 'real sociedad b'
]

# Métricas y pesos
metricas_pesos = {
    'key_passes_avg': 0.06, 'pre_assist_avg': 0.06, 'smart_passes_avg': 0.06,
    'pass_to_penalty_area_avg': 0.05, 'accurate_pass_to_penalty_area_percent': 0.05,
    'shot_assists_avg': 0.05, 'touch_in_box_avg': 0.07,
    'dribbles_avg': 0.05, 'successful_dribbles_percent': 0.05,
    'progressive_run_avg': 0.05, 'progressive_pass_avg': 0.05,
    'successful_progressive_pass_percent': 0.05,
    'xg_shot_avg': 0.08, 'shots_avg': 0.05, 'shots_on_target_percent': 0.05,
    'non_penalty_goal_avg': 0.05, 'assists_avg': 0.05, 'xg_assist_avg': 0.05,
    'minutes_on_field': 0.01
}

# Función para calcular ranking
def calcular_ranking(df, clubes):
    subset = df[
        (df['League'].str.lower() == 'primera rfef') &
        (df['position_gen_ESP1'].str.lower() == 'delantero') &
        (df['minutes_on_field'] >= 1000) &
        (df['last_club_name'].str.lower().isin(clubes))
    ].copy()

    for col, peso in metricas_pesos.items():
        ascending = True if col == 'minutes_on_field' else False
        subset[f'{col}_rank'] = subset[col].rank(pct=True, ascending=ascending)

    subset['impacto_ofensivo'] = sum(
        subset[f'{col}_rank'] * peso for col, peso in metricas_pesos.items()
    ).round(3)

    return subset.sort_values('impacto_ofensivo', ascending=False).head(25)

top_25 = calcular_ranking(df, [c.lower() for c in clubes_validos])
top_25_filial = calcular_ranking(df, [c.lower() for c in clubes_filiales])

# Clase PDF personalizada
class PDF(FPDF):
    def __init__(self):
        super().__init__()
        self.set_auto_page_break(auto=True, margin=15)

    def header(self):
        if hasattr(self, 'title'):
            self.set_font("Arial", "B", 14)
            self.cell(0, 10, self.title, ln=True, align="C")
            self.ln(5)

    def add_ranking_page(self, df_ranking, title):
        self.add_page()
        self.title = title

        for _, row in df_ranking.iterrows():
            # Controlar espacio suficiente
            if self.get_y() > 260:
                self.add_page()

            y_start = self.get_y()
            self.set_draw_color(200, 200, 200)
            self.line(10, y_start, 200, y_start)

            # Imagen del jugador (desde URL)
            if pd.notna(row['image']) and isinstance(row['image'], str) and row['image'].startswith('http'):
                try:
                    response = requests.get(row['image'], stream=True, timeout=5)
                    if response.status_code == 200:
                        temp_img = BytesIO(response.content)
                        self.image(temp_img, x=12, y=y_start + 2, w=12, h=12)
                except:
                    pass

            self.set_xy(26, y_start + 2)
            self.set_font("Arial", "B", 10)
            self.cell(0, 5, str(row['name']), ln=1)

            self.set_x(26)
            self.set_font("Arial", "", 9)
            self.cell(0, 5, f"Índice: {row['impacto_ofensivo']:.3f} | Edad: {int(row['age'])} | Equipo 24/25: {row['last_club_name']}", ln=1)

            self.set_x(26)
            self.cell(0, 5, f"Equipo actual: {row['current_team_name']}", ln=1)

            # Logo del equipo actual (desde URL)
            if pd.notna(row['current_team_logo']) and isinstance(row['current_team_logo'], str) and row['current_team_logo'].startswith('http'):
                try:
                    response = requests.get(row['current_team_logo'], stream=True, timeout=5)
                    if response.status_code == 200:
                        temp_logo = BytesIO(response.content)
                        self.image(temp_logo, x=170, y=y_start + 3, w=15, h=10)
                except:
                    pass

            self.set_y(y_start + 17)
            self.set_draw_color(200, 200, 200)
            self.line(10, self.get_y(), 200, self.get_y())
            self.ln(3)

    def add_metricas_page(self):
        self.add_page()
        self.set_font("Arial", "B", 14)
        self.cell(0, 10, "Métricas utilizadas para el índice", ln=True, align="C")
        self.ln(5)

        def add_section(title, items):
            self.set_font("Arial", "B", 12)
            self.cell(0, 8, title, ln=True)
            self.set_font("Arial", "", 11)
            for item in items:
                self.cell(0, 8, f"- {item}", ln=True)
            self.ln(3)

        add_section("1. Participación en generación de ocasiones", [
            "key_passes_avg", "pre_assist_avg", "smart_passes_avg",
            "pass_to_penalty_area_avg", "accurate_pass_to_penalty_area_percent",
            "shot_assists_avg", "touch_in_box_avg"
        ])

        add_section("2. Desborde y progresión", [
            "dribbles_avg", "successful_dribbles_percent", "progressive_run_avg",
            "progressive_pass_avg", "successful_progressive_pass_percent"
        ])

        add_section("3. Aporte ofensivo directo", [
            "xg_shot_avg", "shots_avg", "shots_on_target_percent",
            "non_penalty_goal_avg", "assists_avg", "xg_assist_avg"
        ])

        add_section("4. Eficiencia con pocos minutos", [
            "minutes_on_field", "goals_avg por 90'", "xg_shot_avg por 90'",
            "assists_avg por 90'", "pre_assist_avg por 90'"
        ])

# Crear PDF
pdf = PDF()
pdf.add_ranking_page(top_25, "Top 25 Delanteros - No Filiales")
pdf.add_ranking_page(top_25_filial, "Top 25 Delanteros - Filiales")
pdf.add_metricas_page()
pdf.output("Ranking_Delanteros_FPDF.pdf")

print("✅ PDF generado correctamente.")

✅ PDF generado correctamente.


In [25]:
import pandas as pd
from fpdf import FPDF
import requests
from PIL import Image
from io import BytesIO

# Cargar CSV
ruta_csv = '/Users/macmontxinho/Desktop/Teams/Wyscout/data/Jugadores.csv'
df = pd.read_csv(ruta_csv, sep=';')

# Clubes
clubes_validos = [
    'intercity sj d\' alacant', 'ibiza', 'marbella', 'gimnàstic tarragona', 'fc andorra',
    'lugo', 'real murcia', 'zamora', 'unionistas de salamanca', 'cultural leonesa',
    'alcorcón', 'mérida ad', 'arenteiro', 'alcoyano', 'antequera', 'ceuta', 'fuenlabrada',
    'ponferradina', 'ourense cf', 'sestao river', 'real unión', 'algeciras', 'hércules',
    'recreativo huelva', 'tarazona', 'atlético sanluqueño', 'yeclano', 'amorebieta',
    'barakaldo', 'gimnástica segoviana'
]
clubes_filiales = [
    'atlético madrid b', 'barcelona atlètic', 'villarreal b', 'real madrid castilla',
    'osasuna promesas', 'celta fortuna', 'sevilla atlético', 'betis deportivo',
    'bilbao athletic', 'celta de vigo b', 'osasuna b', 'real sociedad b'
]

# Métricas y pesos
metricas_pesos = {
    'key_passes_avg': 0.06, 'pre_assist_avg': 0.06, 'smart_passes_avg': 0.06,
    'pass_to_penalty_area_avg': 0.05, 'accurate_pass_to_penalty_area_percent': 0.05,
    'shot_assists_avg': 0.05, 'touch_in_box_avg': 0.07, 'dribbles_avg': 0.05,
    'successful_dribbles_percent': 0.05, 'progressive_run_avg': 0.05,
    'progressive_pass_avg': 0.05, 'successful_progressive_pass_percent': 0.05,
    'xg_shot_avg': 0.08, 'shots_avg': 0.05, 'shots_on_target_percent': 0.05,
    'non_penalty_goal_avg': 0.05, 'assists_avg': 0.05, 'xg_assist_avg': 0.05,
    'minutes_on_field': 0.01
}

# Calcular ranking
def calcular_ranking(df, clubes):
    subset = df[
        (df['League'].str.lower() == 'primera rfef') &
        (df['position_gen_ESP1'].str.lower() == 'delantero') &
        (df['minutes_on_field'] >= 1000) &
        (df['last_club_name'].str.lower().isin(clubes))
    ].copy()

    subset = subset.dropna(subset=metricas_pesos.keys())

    for col, peso in metricas_pesos.items():
        ascending = True if col == 'minutes_on_field' else False
        subset[f'{col}_rank'] = subset[col].rank(pct=True, ascending=ascending)

    subset['impacto_ofensivo'] = sum(
        subset[f'{col}_rank'] * peso for col, peso in metricas_pesos.items()
    ).round(3)

    return subset.sort_values('impacto_ofensivo', ascending=False).head(25)

top_25 = calcular_ranking(df, [c.lower() for c in clubes_validos])
top_25_filial = calcular_ranking(df, [c.lower() for c in clubes_filiales])

# Clase PDF
class PDF(FPDF):
    def __init__(self):
        super().__init__()
        self.set_auto_page_break(auto=True, margin=15)

    def header(self):
        if hasattr(self, 'title'):
            self.set_font("Arial", "B", 14)
            self.cell(0, 10, self.title, ln=True, align="C")
            self.ln(5)

    def add_ranking_page(self, df_ranking, title):
        self.add_page()
        self.title = title

        for i, (_, row) in enumerate(df_ranking.iterrows(), 1):
            if self.get_y() > 260:
                self.add_page()

            y_start = self.get_y()
            self.set_draw_color(180, 180, 180)
            self.line(10, y_start, 200, y_start)

            # Imagen del jugador
            if pd.notna(row['image']) and row['image'].startswith('http'):
                try:
                    r = requests.get(row['image'], timeout=5)
                    img = BytesIO(r.content)
                    self.image(img, x=12, y=y_start + 2, w=12, h=12)
                except:
                    pass

            self.set_xy(26, y_start + 2)
            self.set_font("Arial", "B", 10)
            self.cell(0, 5, f"{i}. {row['name']}", ln=1)

            self.set_x(26)
            self.set_font("Arial", "", 9)
            self.cell(0, 5, f"Índice: {row['impacto_ofensivo']:.3f} | Edad: {int(row['age'])} | Equipo 24/25: {row['last_club_name']}", ln=1)

            self.set_x(26)
            self.cell(0, 5, f"Equipo actual: {row['current_team_name']}", ln=1)

            # Logo del equipo actual
            if pd.notna(row['current_team_logo']) and row['current_team_logo'].startswith('http'):
                try:
                    r = requests.get(row['current_team_logo'], timeout=5)
                    logo = BytesIO(r.content)
                    self.image(logo, x=170, y=y_start + 3, w=15, h=10)
                except:
                    pass

            self.set_y(y_start + 17)
            self.line(10, self.get_y(), 200, self.get_y())
            self.ln(3)

    def add_metricas_page(self):
        self.add_page()
        self.set_font("Arial", "B", 14)
        self.cell(0, 10, "Métricas utilizadas para el índice", ln=True, align="C")
        self.ln(5)

        def add_section(title, items):
            self.set_font("Arial", "B", 12)
            self.cell(0, 8, title, ln=True)
            self.set_font("Arial", "", 11)
            for item in items:
                self.cell(0, 8, f"- {item}", ln=True)
            self.ln(3)

        add_section("1. Participación en generación de ocasiones", [
            "key_passes_avg", "pre_assist_avg", "smart_passes_avg",
            "pass_to_penalty_area_avg", "accurate_pass_to_penalty_area_percent",
            "shot_assists_avg", "touch_in_box_avg"
        ])
        add_section("2. Desborde y progresión", [
            "dribbles_avg", "successful_dribbles_percent", "progressive_run_avg",
            "progressive_pass_avg", "successful_progressive_pass_percent"
        ])
        add_section("3. Aporte ofensivo directo", [
            "xg_shot_avg", "shots_avg", "shots_on_target_percent",
            "non_penalty_goal_avg", "assists_avg", "xg_assist_avg"
        ])
        add_section("4. Eficiencia con pocos minutos", [
            "minutes_on_field", "goals_avg por 90'", "xg_shot_avg por 90'",
            "assists_avg por 90'", "pre_assist_avg por 90'"
        ])

# Generar PDF
pdf = PDF()
pdf.add_ranking_page(top_25, "Top 25 Delanteros - No Filiales")
pdf.add_ranking_page(top_25_filial, "Top 25 Delanteros - Filiales")
pdf.add_metricas_page()
pdf.output("Ranking_Delanteros_FPDF.pdf")

print("✅ PDF generado correctamente.")

✅ PDF generado correctamente.


## CON PORTADA

In [30]:
import pandas as pd
from fpdf import FPDF
import requests
from PIL import Image
from io import BytesIO

# Cargar CSV
ruta_csv = '/Users/macmontxinho/Desktop/Teams/Wyscout/data/Jugadores.csv'
df = pd.read_csv(ruta_csv, sep=';')

# Clubes
clubes_validos = [
    'intercity sj d\' alacant', 'ibiza', 'marbella', 'gimnàstic tarragona', 'fc andorra',
    'lugo', 'real murcia', 'zamora', 'unionistas de salamanca', 'cultural leonesa',
    'alcorcón', 'mérida ad', 'arenteiro', 'alcoyano', 'antequera', 'ceuta', 'fuenlabrada',
    'ponferradina', 'ourense cf', 'sestao river', 'real unión', 'algeciras', 'hércules',
    'recreativo huelva', 'tarazona', 'atlético sanluqueño', 'yeclano', 'amorebieta',
    'barakaldo', 'gimnástica segoviana'
]
clubes_filiales = [
    'atlético madrid b', 'barcelona atlètic', 'villarreal b', 'real madrid castilla',
    'osasuna promesas', 'celta fortuna', 'sevilla atlético', 'betis deportivo',
    'bilbao athletic', 'celta de vigo b', 'osasuna b', 'real sociedad b'
]

# Métricas y pesos
metricas_pesos = {
    'key_passes_avg': 0.06, 'pre_assist_avg': 0.06, 'smart_passes_avg': 0.06,
    'pass_to_penalty_area_avg': 0.05, 'accurate_pass_to_penalty_area_percent': 0.05,
    'shot_assists_avg': 0.05, 'touch_in_box_avg': 0.07, 'dribbles_avg': 0.05,
    'successful_dribbles_percent': 0.05, 'progressive_run_avg': 0.05,
    'progressive_pass_avg': 0.05, 'successful_progressive_pass_percent': 0.05,
    'xg_shot_avg': 0.08, 'shots_avg': 0.05, 'shots_on_target_percent': 0.05,
    'non_penalty_goal_avg': 0.05, 'assists_avg': 0.05, 'xg_assist_avg': 0.05,
    'minutes_on_field': 0.01
}

# Calcular ranking
def calcular_ranking(df, clubes):
    subset = df[
        (df['League'].str.lower() == 'primera rfef') &
        (df['position_gen_ESP1'].str.lower() == 'delantero') &
        (df['minutes_on_field'] >= 1000) &
        (df['last_club_name'].str.lower().isin(clubes))
    ].copy()

    subset = subset.dropna(subset=metricas_pesos.keys())

    for col, peso in metricas_pesos.items():
        ascending = True if col == 'minutes_on_field' else False
        subset[f'{col}_rank'] = subset[col].rank(pct=True, ascending=ascending)

    subset['impacto_ofensivo'] = sum(
        subset[f'{col}_rank'] * peso for col, peso in metricas_pesos.items()
    ).round(3)

    return subset.sort_values('impacto_ofensivo', ascending=False).head(25)

top_25 = calcular_ranking(df, [c.lower() for c in clubes_validos])
top_25_filial = calcular_ranking(df, [c.lower() for c in clubes_filiales])

# Clase PDF
class PDF(FPDF):
    def __init__(self):
        super().__init__()
        self.set_auto_page_break(auto=True, margin=15)

    def portada(self, path_logo):
        self.add_page()
        self.set_font("Arial", "B", 10)
        self.cell(0, 10, "Secretaría Técnica de Unionistas de Salamanca CF", ln=True, align="C")
        self.ln(50)

        # Logo centrado
        if path_logo:
            self.image(path_logo, x=(210-60)/2, y=50, w=60)
            self.ln(60)

        self.set_font("Arial", "B", 18)
        self.cell(0, 10, "Informe Top 25 delanteros de 1RFEF", ln=True, align="C")
        self.ln(10)
        self.set_font("Arial", "", 14)
        self.cell(0, 10, "Separado por No Filiales y Filiales", ln=True, align="C")

    def header(self):
        if hasattr(self, 'title') and self.page_no() > 1:
            self.set_font("Arial", "B", 14)
            self.cell(0, 10, self.title, ln=True, align="C")
            self.ln(5)

    def add_ranking_page(self, df_ranking, title):
        self.title = title
        self.add_page()
        

        for i, (_, row) in enumerate(df_ranking.iterrows(), 1):
            # Altura estimada por jugador
            altura_jugador = 22
            
            # Si no hay espacio suficiente, nueva página
            if self.get_y() + altura_jugador > 270:
                self.add_page()

            y_start = self.get_y()

            # Línea superior
            self.set_draw_color(180, 180, 180)
            self.line(10, y_start, 200, y_start)

            # Imagen del jugador
            if pd.notna(row['image']) and row['image'].startswith('http'):
                try:
                    r = requests.get(row['image'], timeout=5)
                    if r.status_code == 200:
                        with tempfile.NamedTemporaryFile(delete=False, suffix=".png") as tmp_img:
                            tmp_img.write(r.content)
                            tmp_img.flush()
                            self.image(tmp_img.name, x=12, y=y_start + 2, w=12, h=12)
                except:
                    pass

            # Texto
            self.set_xy(26, y_start + 2)
            self.set_font("Arial", "B", 10)
            self.cell(0, 5, f"{i}. {row['name']}", ln=1)

            self.set_x(26)
            self.set_font("Arial", "", 9)
            self.cell(0, 5, f"Índice: {row['impacto_ofensivo']:.3f} | Edad: {int(row['age'])} | Equipo 24/25: {row['last_club_name']}", ln=1)

            self.set_x(26)
            self.cell(0, 5, f"Equipo actual: {row['current_team_name']}", ln=1)

            # Línea inferior
            self.set_y(y_start + altura_jugador - 5)
            self.line(10, self.get_y(), 200, self.get_y())
            self.ln(3)

    def add_metricas_page(self):
        self.title = "Métricas Seleccionadas"
        self.add_page()
        self.set_font("Arial", "B", 14)
        self.cell(0, 10, "Métricas utilizadas para el índice", ln=True, align="C")
        self.ln(5)

        def add_section(title, items):
            self.set_font("Arial", "B", 12)
            self.cell(0, 8, title, ln=True)
            self.set_font("Arial", "", 11)
            for item in items:
                self.cell(0, 8, f"- {item}", ln=True)
            self.ln(3)

        add_section("1. Participación en generación de ocasiones", [
            "key_passes_avg", "pre_assist_avg", "smart_passes_avg",
            "pass_to_penalty_area_avg", "accurate_pass_to_penalty_area_percent",
            "shot_assists_avg", "touch_in_box_avg"
        ])
        add_section("2. Desborde y progresión", [
            "dribbles_avg", "successful_dribbles_percent", "progressive_run_avg",
            "progressive_pass_avg", "successful_progressive_pass_percent"
        ])
        add_section("3. Aporte ofensivo directo", [
            "xg_shot_avg", "shots_avg", "shots_on_target_percent",
            "non_penalty_goal_avg", "assists_avg", "xg_assist_avg"
        ])
        add_section("4. Eficiencia con pocos minutos", [
            "minutes_on_field", "goals_avg por 90'", "xg_shot_avg por 90'",
            "assists_avg por 90'", "pre_assist_avg por 90'"
        ])

# Ruta logo portada
ruta_logo = "/Users/macmontxinho/Desktop/Teams/Unionistas/Imagenes/IMG_2027-removebg-preview.png"

# Generar PDF
pdf = PDF()
pdf.portada(ruta_logo)
pdf.add_ranking_page(top_25, "Top 25 Delanteros - No Filiales")
pdf.add_ranking_page(top_25_filial, "Top 25 Delanteros - Filiales")
pdf.add_metricas_page()
pdf.output("Ranking_Delanteros_FPDF.pdf")

print("✅ PDF generado correctamente.")

✅ PDF generado correctamente.


In [ ]:
import pandas as pd
from fpdf import FPDF
import requests
from PIL import Image
from io import BytesIO
import tempfile

# === CARGA DE DATOS ===
ruta_csv = '/Users/macmontxinho/Desktop/Teams/Wyscout/data/Jugadores.csv'
df = pd.read_csv(ruta_csv, sep=';')

clubes_validos = [
    'intercity sj d\' alacant', 'ibiza', 'marbella', 'gimnàstic tarragona', 'fc andorra',
    'lugo', 'real murcia', 'zamora', 'unionistas de salamanca', 'cultural leonesa',
    'alcorcón', 'mérida ad', 'arenteiro', 'alcoyano', 'antequera', 'ceuta', 'fuenlabrada',
    'ponferradina', 'ourense cf', 'sestao river', 'real unión', 'algeciras', 'hércules',
    'recreativo huelva', 'tarazona', 'atlético sanluqueño', 'yeclano', 'amorebieta',
    'barakaldo', 'gimnástica segoviana'
]
clubes_filiales = [
    'atlético madrid b', 'barcelona atlètic', 'villarreal b', 'real madrid castilla',
    'osasuna promesas', 'celta fortuna', 'sevilla atlético', 'betis deportivo',
    'bilbao athletic', 'celta de vigo b', 'osasuna b', 'real sociedad b'
]

metricas_pesos = {
    'key_passes_avg': 0.06, 'pre_assist_avg': 0.06, 'smart_passes_avg': 0.06,
    'pass_to_penalty_area_avg': 0.05, 'accurate_pass_to_penalty_area_percent': 0.05,
    'shot_assists_avg': 0.05, 'touch_in_box_avg': 0.07, 'dribbles_avg': 0.05,
    'successful_dribbles_percent': 0.05, 'progressive_run_avg': 0.05,
    'progressive_pass_avg': 0.05, 'successful_progressive_pass_percent': 0.05,
    'xg_shot_avg': 0.08, 'shots_avg': 0.05, 'shots_on_target_percent': 0.05,
    'non_penalty_goal_avg': 0.05, 'assists_avg': 0.05, 'xg_assist_avg': 0.05,
    'minutes_on_field': 0.01
}

def calcular_ranking(df, clubes):
    subset = df[
        (df['League'].str.lower() == 'primera rfef') &
        (df['position_gen_ESP1'].str.lower() == 'delantero') &
        (df['minutes_on_field'] >= 1000) &
        (df['last_club_name'].str.lower().isin(clubes))
    ].copy()

    subset = subset.dropna(subset=metricas_pesos.keys())

    for col, peso in metricas_pesos.items():
        ascending = True if col == 'minutes_on_field' else False
        subset[f'{col}_rank'] = subset[col].rank(pct=True, ascending=ascending)

    subset['impacto_ofensivo'] = sum(
        subset[f'{col}_rank'] * peso for col, peso in metricas_pesos.items()
    ).round(3)

    return subset.sort_values('impacto_ofensivo', ascending=False).head(25)

top_25 = calcular_ranking(df, [c.lower() for c in clubes_validos])
top_25_filial = calcular_ranking(df, [c.lower() for c in clubes_filiales])

# === CLASE PDF ===
class PDF(FPDF):
    def __init__(self):
        super().__init__()
        self.set_auto_page_break(auto=True, margin=15)

    def portada(self, path_logo):
        self.add_page()
        self.set_font("Arial", "B", 10)
        self.cell(0, 10, "Secretaría Técnica de Unionistas de Salamanca CF", ln=True, align="C")
        self.ln(50)
        if path_logo:
            self.image(path_logo, x=(210-60)/2, y=50, w=60)
        self.set_y(120)
        self.set_font("Arial", "B", 18)
        self.cell(0, 10, "Informe Top 25 delanteros de 1RFEF", ln=True, align="C")
        self.ln(10)
        self.set_font("Arial", "", 14)
        self.cell(0, 10, "Separado por No Filiales y Filiales", ln=True, align="C")

    def header(self):
        if hasattr(self, 'title') and self.page_no() > 1:
            self.set_font("Arial", "B", 14)
            self.cell(0, 10, self.title, ln=True, align="C")
            self.ln(5)

    def add_ranking_page(self, df_ranking, title):
        self.title = title
        self.add_page()

        for i, (_, row) in enumerate(df_ranking.iterrows(), 1):
            altura_jugador = 22
            if self.get_y() + altura_jugador > 270:
                self.add_page()

            y_start = self.get_y()

            # Línea superior
            self.set_draw_color(180, 180, 180)
            self.line(10, y_start, 200, y_start)

            # Imagen jugador
            if pd.notna(row['image']) and row['image'].startswith('http'):
                try:
                    r = requests.get(row['image'], timeout=5)
                    if r.status_code == 200:
                        with tempfile.NamedTemporaryFile(delete=False, suffix=".png") as tmp_img:
                            tmp_img.write(r.content)
                            tmp_img.flush()
                            self.image(tmp_img.name, x=12, y=y_start + 2, w=12, h=12)
                except:
                    pass

            # Texto
            self.set_xy(26, y_start + 2)
            self.set_font("Arial", "B", 10)
            self.cell(0, 5, f"{i}. {row['name']}", ln=1)

            self.set_x(26)
            self.set_font("Arial", "", 9)
            self.cell(0, 5, f"Índice: {row['impacto_ofensivo']:.3f} | Edad: {int(row['age'])} | Equipo 24/25: {row['last_club_name']}", ln=1)

            self.set_x(26)
            self.cell(0, 5, f"Equipo actual: {row['current_team_name']}", ln=1)

            # Línea inferior
            self.set_y(y_start + altura_jugador - 5)
            self.line(10, self.get_y(), 200, self.get_y())
            self.ln(3)

        def add_metricas_page(self):
            self.title = "Métricas Seleccionadas"
            self.add_page()
            self.set_font("Arial", "B", 14)
            self.cell(0, 10, "Métricas utilizadas para el índice", ln=True, align="C")
            self.ln(5)

            def add_section(title, items):
                self.set_font("Arial", "B", 12)
                self.cell(0, 8, title, ln=True)
                self.set_font("Arial", "", 11)
                for item in items:
                    self.cell(0, 8, f"- {item}", ln=True)
                self.ln(3)

            add_section("1. Participación en generación de ocasiones", [
                "key_passes_avg", "pre_assist_avg", "smart_passes_avg",
                "pass_to_penalty_area_avg", "accurate_pass_to_penalty_area_percent",
                "shot_assists_avg", "touch_in_box_avg"
            ])
            add_section("2. Desborde y progresión", [
                "dribbles_avg", "successful_dribbles_percent", "progressive_run_avg",
                "progressive_pass_avg", "successful_progressive_pass_percent"
            ])
            add_section("3. Aporte ofensivo directo", [
                "xg_shot_avg", "shots_avg", "shots_on_target_percent",
                "non_penalty_goal_avg", "assists_avg", "xg_assist_avg"
            ])
            add_section("4. Eficiencia con pocos minutos", [
                "minutes_on_field", "goals_avg por 90'", "xg_shot_avg por 90'",
                "assists_avg por 90'", "pre_assist_avg por 90'"
            ])

        def add_explicacion_metricas(self, path_img_tabla=None):
            self.title = "Explicación del índice y su impacto"
            self.add_page()
            
            self.set_font("Arial", "B", 13)
            self.cell(0, 10, "¿Qué tipo de delanteros se ven favorecidos o perjudicados por este índice?", ln=True)
            self.ln(5)

            self.set_font("Arial", "", 10)
            texto = (
                "✅ El índice favorece a:\n"
                "• Delanteros creativos o de apoyo (segunda punta, falso 9)\n"
                "• Delanteros móviles, con llegada, que rematan y se desmarcan\n"
                "• Delanteros versátiles que combinan creación, asistencia y gol\n\n"
                "❌ El índice penaliza a:\n"
                "• Delanteros estáticos o rematadores puros con poca participación\n"
                "• Extremos reconvertidos si no aportan en finalización ni generación\n\n"
                "➡️ Este índice premia la participación activa y constante en todas las fases ofensivas."
            )
            self.multi_cell(0, 6, texto)
            self.ln(5)

            if path_img_tabla and os.path.exists(path_img_tabla):
                self.image(path_img_tabla, x=15, w=180)

        

# === EXPORTAR PDF ===
ruta_logo = "/Users/macmontxinho/Desktop/Teams/Unionistas/Imagenes/IMG_2027-removebg-preview.png"

pdf = PDF()
pdf.portada(ruta_logo)
pdf.add_ranking_page(top_25, "Top 25 Delanteros - No Filiales")
pdf.add_ranking_page(top_25_filial, "Top 25 Delanteros - Filiales")
pdf.add_metricas_page()
pdf.output("Ranking_Delanteros_FPDF.pdf")

print("✅ PDF generado correctamente.")

✅ PDF generado correctamente.


In [52]:
import pandas as pd
from fpdf import FPDF
import requests
from io import BytesIO
import os
import matplotlib.pyplot as plt
import tempfile

# --------------------
# Cargar CSV y datos
# --------------------
ruta_csv = '/Users/macmontxinho/Desktop/Teams/Wyscout/data/Jugadores.csv'
df = pd.read_csv(ruta_csv, sep=';')

clubes_validos = [
    'intercity sj d\' alacant', 'ibiza', 'marbella', 'gimnàstic tarragona', 'fc andorra',
    'lugo', 'real murcia', 'zamora', 'unionistas de salamanca', 'cultural leonesa',
    'alcorcón', 'mérida ad', 'arenteiro', 'alcoyano', 'antequera', 'ceuta', 'fuenlabrada',
    'ponferradina', 'ourense cf', 'sestao river', 'real unión', 'algeciras', 'hércules',
    'recreativo huelva', 'tarazona', 'atlético sanluqueño', 'yeclano', 'amorebieta',
    'barakaldo', 'gimnástica segoviana']

clubes_filiales = [
    'atlético madrid b', 'barcelona atlètic', 'villarreal b', 'real madrid castilla',
    'osasuna promesas', 'celta fortuna', 'sevilla atlético', 'betis deportivo',
    'bilbao athletic', 'celta de vigo b', 'osasuna b', 'real sociedad b']

metricas_pesos = {
    # Finalización (más peso)
    'non_penalty_goal_avg': 0.15,
    'xg_shot_avg': 0.10,
    'shots_avg': 0.07,
    'shots_on_target_percent': 0.06,
    'head_goals_avg': 0.04,  # 🆕 añadida

    # Asistencia y creación directa
    'assists_avg': 0.08,
    'xg_assist_avg': 0.07,
    'touch_in_box_avg': 0.07,
    'shot_assists_avg': 0.05,
    'key_passes_avg': 0.05,

    # Resto de aportes ofensivos
    'dribbles_avg': 0.04,
    'successful_dribbles_percent': 0.03,
    'progressive_run_avg': 0.03,
    'progressive_pass_avg': 0.03,
    'successful_progressive_pass_percent': 0.02,
    'pre_assist_avg': 0.03,
    'smart_passes_avg': 0.03,
    'pass_to_penalty_area_avg': 0.03,
    'accurate_pass_to_penalty_area_percent': 0.03
}
def calcular_ranking(df, clubes):
    subset = df[
        (df['League'].str.lower() == 'primera rfef') &
        (df['position_gen_ESP1'].str.lower() == 'delantero') &
        (df['minutes_on_field'] >= 1000) &
        (df['last_club_name'].str.lower().isin(clubes))
    ].copy()

    subset = subset.dropna(subset=metricas_pesos.keys())

    for col, peso in metricas_pesos.items():
        ascending = True if col == 'minutes_on_field' else False
        subset[f'{col}_rank'] = subset[col].rank(pct=True, ascending=ascending)

    subset['impacto_ofensivo'] = sum(
        subset[f'{col}_rank'] * peso for col, peso in metricas_pesos.items()
    ).round(3)

    return subset.sort_values('impacto_ofensivo', ascending=False).head(25)

top_25 = calcular_ranking(df, [c.lower() for c in clubes_validos])
top_25_filial = calcular_ranking(df, [c.lower() for c in clubes_filiales])

class PDF(FPDF):
    def __init__(self):
        super().__init__()
        self.set_auto_page_break(auto=True, margin=15)

    def portada(self, path_logo):
        self.add_page()
        self.set_font("Arial", "B", 10)
        self.cell(0, 10, "Secretaría Técnica de Unionistas de Salamanca CF", ln=True, align="C")
        self.ln(50)
        if path_logo and os.path.exists(path_logo):
            self.image(path_logo, x=(210 - 60) / 2, y=50, w=60)
            self.ln(60)
        self.set_font("Arial", "B", 18)
        self.cell(0, 10, "Informe Top 25 delanteros de 1RFEF", ln=True, align="C")
        self.ln(10)
        self.set_font("Arial", "", 14)
        self.cell(0, 10, "Separado por No Filiales y Filiales", ln=True, align="C")

    def header(self):
        if hasattr(self, 'title') and self.page_no() > 1:
            self.set_font("Arial", "B", 14)
            self.cell(0, 10, self.title, ln=True, align="C")
            self.ln(5)

    def footer(self):
        if self.page_no() == 1:
            return  # Omitir footer en portada

        self.set_y(-15)

        # Texto izquierda
        self.set_font('Arial', '', 8)
        self.set_x(10)
        self.cell(60, 10, "Informe realizado por: Ramón Codesido", align='L')

        # Número de página centrado
        self.set_x(90)  # Aproximadamente centrado para un ancho de 30-40px
        self.set_font('Arial', 'I', 8)
        self.cell(30, 10, f'Página {self.page_no() - 1}', align='C')

        # Logo derecha
        path_footer_logo = "/Users/macmontxinho/Desktop/Teams/Unionistas/Imagenes/Escudo_Of.png"
        if os.path.exists(path_footer_logo):
            self.image(path_footer_logo, x=180, y=self.get_y(), w=14)

    def add_ranking_page(self, df_ranking, title):
        self.title = title
        self.add_page()
        for i, (_, row) in enumerate(df_ranking.iterrows(), 1):
            altura_jugador = 22
            if self.get_y() + altura_jugador > 270:
                self.add_page()
            y_start = self.get_y()
            self.set_draw_color(180, 180, 180)
            self.line(10, y_start, 200, y_start)
            if pd.notna(row['image']) and row['image'].startswith('http'):
                try:
                    r = requests.get(row['image'], timeout=5)
                    if r.status_code == 200:
                        with tempfile.NamedTemporaryFile(delete=False, suffix=".png") as tmp_img:
                            tmp_img.write(r.content)
                            tmp_img.flush()
                            self.image(tmp_img.name, x=12, y=y_start + 2, w=12, h=12)
                except:
                    pass
            self.set_xy(26, y_start + 2)
            self.set_font("Arial", "B", 10)
            self.cell(0, 5, f"{i}. {row['name']}", ln=1)
            self.set_x(26)
            self.set_font("Arial", "", 9)
            self.cell(0, 5, f"Índice: {row['impacto_ofensivo']:.3f} | Edad: {int(row['age'])} | Equipo 24/25: {row['last_club_name']}", ln=1)
            self.set_x(26)
            self.cell(0, 5, f"Equipo actual: {row['current_team_name']}", ln=1)
            self.set_y(y_start + altura_jugador - 5)
            self.line(10, self.get_y(), 200, self.get_y())
            self.ln(3)

    def add_metricas_page(self):
        self.title = "Métricas Seleccionadas"
        self.add_page()
        self.set_font("Arial", "B", 14)
        self.cell(0, 10, "Métricas utilizadas para el índice", ln=True, align="C")
        self.ln(5)

        def add_section(title, items):
            self.set_font("Arial", "B", 12)
            self.cell(0, 8, title, ln=True)
            self.set_font("Arial", "", 11)
            for item in items:
                self.cell(0, 8, f"- {item}", ln=True)
            self.ln(3)

        add_section("1. Finalización (mayor peso)", [
            "non_penalty_goal_avg", "xg_shot_avg", "shots_avg", 
            "shots_on_target_percent", "head_goals_avg"
        ])
        add_section("2. Asistencia y creación directa", [
            "assists_avg", "xg_assist_avg", "touch_in_box_avg", 
            "shot_assists_avg", "key_passes_avg"
        ])
        add_section("3. Construcción ofensiva", [
            "pre_assist_avg", "smart_passes_avg", "pass_to_penalty_area_avg", 
            "accurate_pass_to_penalty_area_percent"
        ])
        add_section("4. Progresión y desborde", [
            "dribbles_avg", "successful_dribbles_percent", 
            "progressive_run_avg", "progressive_pass_avg", 
            "successful_progressive_pass_percent"
])

    def add_explicacion_metricas(self, path_img_tabla=None):
        self.title = "Explicación del índice y su impacto"
        self.add_page()
        self.set_font("Arial", "B", 13)
        self.cell(0, 10, "¿Qué tipo de delanteros se ven favorecidos o perjudicados por este índice?", ln=True)
        self.ln(5)
        self.set_font("Arial", "", 10)
        texto = (
            "El índice favorece a:\n"
            "- Delanteros con alta producción goleadora (9 de área, rematadores)\n"
            "- Delanteros móviles con capacidad de finalizar y asistir\n"
            "- Perfiles que combinan finalización, presencia en el área y creación\n\n"
            "El índice penaliza a:\n"
            "- Delanteros con poca incidencia en el remate o el xG generado\n"
            "- Extremos o atacantes que no aportan cifras ofensivas relevantes\n\n"
            "Este índice premia la capacidad para finalizar jugadas, generar peligro y participar activamente en la fase ofensiva.\n"
            "IMPORTANTE: Se ha aplicado un filtro previo para considerar solo jugadores con mínimo 1.000 minutos disputados en 1RFEF."
        )
        self.multi_cell(0, 6, texto)
        self.ln(5)
        if path_img_tabla and os.path.exists(path_img_tabla):
            self.image(path_img_tabla, x=15, w=180)

def generar_tabla_conclusion(path_salida="tabla_conclusion.png"):
    data = {
        "Tipo de delantero": [
            "Rematador puro (9 clásico)", 
            "Delantero móvil / rupturista", 
            "Delantero versátil (remate + creación)", 
            "Segunda punta / 'enganche'", 
            "Delantero de banda reconvertido"
        ],
        "¿Favorecido?": ["Sí", "Sí", "Mucho", "Depende", "No"],
        "Motivo principal": [
            "Aporta goles, remates y presencia en el área",
            "xG, remate y desmarque, incluso con menos asociación",
            "Contribuye en remate, asistencia y juego",
            "Gran visión y asistencia, pero menos presencia en finalización",
            "Mucho desborde pero bajo impacto en xG o goles"
        ]
    }
    fig, ax = plt.subplots(figsize=(12, 3))
    ax.axis('tight')
    ax.axis('off')
    table = ax.table(cellText=list(zip(*data.values())), colLabels=list(data.keys()), cellLoc='center', loc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(9)
    table.scale(1.3, 1.6)
    for (row, col), cell in table.get_celld().items():
        if row == 0:
            cell.set_fontsize(10)
            cell.set_text_props(weight='bold')
            cell.set_facecolor('#1E2A3B')
            cell.set_text_props(color='white')
        else:
            cell.set_facecolor('#F2F2F2' if row % 2 == 0 else '#FFFFFF')
    plt.savefig(path_salida, bbox_inches='tight', dpi=300)
    plt.close()

# --------------------
# Generar informe
# --------------------
ruta_logo = "/Users/macmontxinho/Desktop/Teams/Unionistas/Imagenes/IMG_2027-removebg-preview.png"
pdf = PDF()
pdf.portada(ruta_logo)
pdf.add_ranking_page(top_25, "Top 25 Delanteros - No Filiales")
pdf.add_ranking_page(top_25_filial, "Top 25 Delanteros - Filiales")
pdf.add_metricas_page()

path_tabla_img = "tabla_conclusion.png"
generar_tabla_conclusion(path_tabla_img)
pdf.add_explicacion_metricas(path_img_tabla=path_tabla_img)
pdf.output("Ranking_Delanteros_FPDF.pdf")

print("PDF generado correctamente.")

PDF generado correctamente.


In [56]:
import pandas as pd
from fpdf import FPDF
import requests
from io import BytesIO
import os
import matplotlib.pyplot as plt
import tempfile

# --------------------
# Cargar CSV y datos
# --------------------
ruta_csv = '/Users/macmontxinho/Desktop/Teams/Wyscout/data/Jugadores.csv'
df = pd.read_csv(ruta_csv, sep=';')

clubes_validos = [
    'intercity sj d\' alacant', 'ibiza', 'marbella', 'gimnàstic tarragona', 'fc andorra',
    'lugo', 'real murcia', 'zamora', 'unionistas de salamanca', 'cultural leonesa',
    'alcorcón', 'mérida ad', 'arenteiro', 'alcoyano', 'antequera', 'ceuta', 'fuenlabrada',
    'ponferradina', 'ourense cf', 'sestao river', 'real unión', 'algeciras', 'hércules',
    'recreativo huelva', 'tarazona', 'atlético sanluqueño', 'yeclano', 'amorebieta',
    'barakaldo', 'gimnástica segoviana']

clubes_filiales = [
    'atlético madrid b', 'barcelona atlètic', 'villarreal b', 'real madrid castilla',
    'osasuna promesas', 'celta fortuna', 'sevilla atlético', 'betis deportivo',
    'bilbao athletic', 'celta de vigo b', 'osasuna b', 'real sociedad b']

metricas_utilizadas = [
    'non_penalty_goal_avg', 'xg_shot_avg', 'shots_avg', 'shots_on_target_percent', 'head_goals_avg',
    'assists_avg', 'xg_assist_avg', 'touch_in_box_avg', 'shot_assists_avg', 'key_passes_avg',
    'dribbles_avg', 'successful_dribbles_percent', 'progressive_run_avg', 'progressive_pass_avg',
    'successful_progressive_pass_percent', 'pre_assist_avg', 'smart_passes_avg',
    'pass_to_penalty_area_avg', 'accurate_pass_to_penalty_area_percent'
]
def calcular_scoring(df, clubes):
    subset = df[
        (df['League'].str.lower() == 'primera rfef') &
        (df['position_gen_ESP1'].str.lower() == 'delantero') &
        (df['minutes_on_field'] >= 1000) &
        (df['last_club_name'].str.lower().isin(clubes))
    ].copy()

    subset = subset.dropna(subset=metricas_utilizadas)

    # Normalización min-max por columna
    for col in metricas_utilizadas:
        min_val = subset[col].min()
        max_val = subset[col].max()
        if min_val != max_val:  # Evita división por cero si todos los valores son iguales
            subset[f'{col}_score'] = (subset[col] - min_val) / (max_val - min_val)
        else:
            subset[f'{col}_score'] = 0  # o 1, o np.nan según tu criterio

    # Media simple de scores
    subset['impacto_ofensivo'] = subset[
        [f'{col}_score' for col in metricas_utilizadas]
    ].mean(axis=1).round(3)

    return subset.sort_values('impacto_ofensivo', ascending=False).head(25)

top_25 = calcular_scoring(df, [c.lower() for c in clubes_validos])
top_25_filial = calcular_scoring(df, [c.lower() for c in clubes_filiales])

class PDF(FPDF):
    def __init__(self):
        super().__init__()
        self.set_auto_page_break(auto=True, margin=15)

    def portada(self, path_logo):
        self.add_page()
        self.set_font("Arial", "B", 10)
        self.cell(0, 10, "Secretaría Técnica de Unionistas de Salamanca CF", ln=True, align="C")
        self.ln(50)
        if path_logo and os.path.exists(path_logo):
            self.image(path_logo, x=(210 - 60) / 2, y=50, w=60)
            self.ln(60)
        self.set_font("Arial", "B", 18)
        self.cell(0, 10, "Informe Top 25 delanteros de 1RFEF", ln=True, align="C")
        self.ln(10)
        self.set_font("Arial", "", 14)
        self.cell(0, 10, "Separado por No Filiales y Filiales", ln=True, align="C")

    def header(self):
        if hasattr(self, 'title') and self.page_no() > 1:
            self.set_font("Arial", "B", 14)
            self.cell(0, 10, self.title, ln=True, align="C")
            self.ln(5)

    def footer(self):
        if self.page_no() == 1:
            return  # Omitir footer en portada

        self.set_y(-15)

        # Texto izquierda
        self.set_font('Arial', '', 8)
        self.set_x(10)
        self.cell(60, 10, "Informe realizado por: Ramón Codesido", align='L')

        # Número de página centrado
        self.set_x(90)  # Aproximadamente centrado para un ancho de 30-40px
        self.set_font('Arial', 'I', 8)
        self.cell(30, 10, f'Página {self.page_no() - 1}', align='C')

        # Logo derecha
        path_footer_logo = "/Users/macmontxinho/Desktop/Teams/Unionistas/Imagenes/Escudo_Of.png"
        if os.path.exists(path_footer_logo):
            self.image(path_footer_logo, x=180, y=self.get_y(), w=14)

    def add_ranking_page(self, df_ranking, title):
        self.title = title
        self.add_page()
        for i, (_, row) in enumerate(df_ranking.iterrows(), 1):
            altura_jugador = 22
            if self.get_y() + altura_jugador > 270:
                self.add_page()
            y_start = self.get_y()
            self.set_draw_color(180, 180, 180)
            self.line(10, y_start, 200, y_start)
            if pd.notna(row['image']) and row['image'].startswith('http'):
                try:
                    r = requests.get(row['image'], timeout=5)
                    if r.status_code == 200:
                        with tempfile.NamedTemporaryFile(delete=False, suffix=".png") as tmp_img:
                            tmp_img.write(r.content)
                            tmp_img.flush()
                            self.image(tmp_img.name, x=12, y=y_start + 2, w=12, h=12)
                except:
                    pass
            self.set_xy(26, y_start + 2)
            self.set_font("Arial", "B", 10)
            self.cell(0, 5, f"{i}. {row['name']}", ln=1)
            self.set_x(26)
            self.set_font("Arial", "", 9)
            self.cell(0, 5, f"Índice: {row['impacto_ofensivo']:.3f} | Edad: {int(row['age'])} | Equipo 24/25: {row['last_club_name']}", ln=1)
            self.set_x(26)
            self.cell(0, 5, f"Equipo actual: {row['current_team_name']}", ln=1)
            self.set_y(y_start + altura_jugador - 5)
            self.line(10, self.get_y(), 200, self.get_y())
            self.ln(3)

    def add_metricas_page(self):
        self.title = "Métricas Seleccionadas"
        self.add_page()
        self.set_font("Arial", "B", 14)
        self.cell(0, 10, "Métricas utilizadas para el índice", ln=True, align="C")
        self.ln(5)

        def add_section(title, items):
            self.set_font("Arial", "B", 12)
            self.cell(0, 8, title, ln=True)
            self.set_font("Arial", "", 11)
            for item in items:
                self.cell(0, 8, f"- {item}", ln=True)
            self.ln(3)

        add_section("1. Finalización (mayor peso)", [
            "non_penalty_goal","non_penalty_goal_avg", "xg_shot_avg", "shots_avg", 
            "shots_on_target_percent", "head_goals_avg"
        ])
        add_section("2. Asistencia y creación directa", [
            "assists", "xg_assist_avg", "touch_in_box_avg", 
            "shot_assists_avg", "key_passes_avg"
        ])
        add_section("3. Construcción ofensiva", [
            "pre_assist_avg", "smart_passes_avg", "pass_to_penalty_area_avg", 
            "accurate_pass_to_penalty_area_percent"
        ])
        add_section("4. Progresión y desborde", [
            "dribbles_avg", "successful_dribbles_percent", 
            "progressive_run_avg", "progressive_pass_avg", 
            "successful_progressive_pass_percent"
])

    def add_explicacion_metricas(self, path_img_tabla=None):
        self.title = "Explicación del índice y su impacto"
        self.add_page()
        self.set_font("Arial", "B", 13)
        self.cell(0, 10, "¿Qué tipo de delanteros se ven favorecidos o perjudicados por este índice?", ln=True)
        self.ln(5)
        self.set_font("Arial", "", 10)
        texto = (
            "El índice favorece a:\n"
            "- Delanteros con alta producción goleadora (9 de área, rematadores)\n"
            "- Delanteros móviles con capacidad de finalizar y asistir\n"
            "- Perfiles que combinan finalización, presencia en el área y creación\n\n"
            "El índice penaliza a:\n"
            "- Delanteros con poca incidencia en el remate o el xG generado\n"
            "- Extremos o atacantes que no aportan cifras ofensivas relevantes\n\n"
            "Este índice premia la capacidad para finalizar jugadas, generar peligro y participar activamente en la fase ofensiva.\n"
            "IMPORTANTE: Se ha aplicado un filtro previo para considerar solo jugadores con mínimo 1.000 minutos disputados en 1RFEF."
        )
        self.multi_cell(0, 6, texto)
        self.ln(5)
        if path_img_tabla and os.path.exists(path_img_tabla):
            self.image(path_img_tabla, x=15, w=180)

def generar_tabla_conclusion(path_salida="tabla_conclusion.png"):
    data = {
        "Tipo de delantero": [
            "Rematador puro (9 clásico)", 
            "Delantero móvil / rupturista", 
            "Delantero versátil (remate + creación)", 
            "Segunda punta / 'enganche'", 
            "Delantero de banda reconvertido"
        ],
        "¿Favorecido?": ["Sí", "Sí", "Mucho", "Depende", "No"],
        "Motivo principal": [
            "Aporta goles, remates y presencia en el área",
            "xG, remate y desmarque, incluso con menos asociación",
            "Contribuye en remate, asistencia y juego",
            "Gran visión y asistencia, pero menos presencia en finalización",
            "Mucho desborde pero bajo impacto en xG o goles"
        ]
    }
    fig, ax = plt.subplots(figsize=(12, 3))
    ax.axis('tight')
    ax.axis('off')
    table = ax.table(cellText=list(zip(*data.values())), colLabels=list(data.keys()), cellLoc='center', loc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(9)
    table.scale(1.3, 1.6)
    for (row, col), cell in table.get_celld().items():
        if row == 0:
            cell.set_fontsize(10)
            cell.set_text_props(weight='bold')
            cell.set_facecolor('#1E2A3B')
            cell.set_text_props(color='white')
        else:
            cell.set_facecolor('#F2F2F2' if row % 2 == 0 else '#FFFFFF')
    plt.savefig(path_salida, bbox_inches='tight', dpi=300)
    plt.close()

# --------------------
# Generar informe
# --------------------
ruta_logo = "/Users/macmontxinho/Desktop/Teams/Unionistas/Imagenes/IMG_2027-removebg-preview.png"
pdf = PDF()
pdf.portada(ruta_logo)
pdf.add_ranking_page(top_25, "Top 25 Delanteros - No Filiales")
pdf.add_ranking_page(top_25_filial, "Top 25 Delanteros - Filiales")
pdf.add_metricas_page()

path_tabla_img = "tabla_conclusion.png"
generar_tabla_conclusion(path_tabla_img)
pdf.add_explicacion_metricas(path_img_tabla=path_tabla_img)
pdf.output("Ranking_Delanteros_FPDF_V1.pdf")

print("PDF generado correctamente.")

PDF generado correctamente.


### 2RFEF

In [61]:
import pandas as pd
from fpdf import FPDF
import requests
from io import BytesIO
import os
import matplotlib.pyplot as plt
import tempfile

# --------------------
# Cargar CSV y datos
# --------------------
ruta_csv = '/Users/macmontxinho/Desktop/Teams/Wyscout/data/Jugadores.csv'
df = pd.read_csv(ruta_csv, sep=';')

clubes_validos = [club.lower() for club in [
    "Bergantiños", "Compostela", "Coruxo", "Escobedo", "Gimnástica Torrelavega",
    "Guijuelo", "Langreo", "Laredo", "Llanera", "Marino de Luanco", "Numancia",
    "Pontevedra", "Real Ávila", "Real Avilés", "Salamanca CF UDS", "Alfaro",
    "Anguiano", "Arenas Club", "Barbastro", "CD Calahorra", "Ejea", "Gernika",
    "Izarra", "SD Logroñés", "Subiza", "Teruel", "Tudelano", "UD Logroñés",
    "Utebo", "Alzira", "Andratx", "Atlético Baleares", "Badalona Futur", "Cornellà",
    "Europa", "Ibiza Islas Pitiusas", "Lleida Esportiu", "Olot", "Peña Deportiva",
    "Sabadell", "Sant Andreu", "Terrassa", "Torrent", "Águilas", "Antoniano",
    "Deportiva Minera", "Don Benito", "Estepona", "Juventud Torremolinos",
    "Linares Deportivo", "Orihuela", "Real Balompédica Linense", "San Fernando",
    "UCAM Murcia", "La Unión Atlético", "Villanovense", "Xerez", "Xerez Deportivo",
    "Atlético Paso", "Cacereño", "CD Coria", "Colonia Moscardó", "Conquense",
    "Guadalajara", "Illescas", "Melilla", "Móstoles", "Navalcarnero",
    "Rayo Majadahonda", "Talavera CF", "UD Sanse", "Unión Adarve", "Unión Sur Yaiza"
]]

clubes_filiales = [club.lower() for club in [
    "Deportivo Fabril", "Racing Santander II", "Real Valladolid Promesas",
    "Deportivo Alavés B", "Eibar B", "Real Sociedad III", "Real Zaragoza B",
    "Elche Illicitano", "Espanyol B", "Mallorca B", "Valencia Mestalla",
    "Almería B", "Cádiz B", "Recreativo Granada", "Getafe B", "Real Madrid C",
    "Tenerife B"
]]

metricas_utilizadas = [
    'non_penalty_goal_avg', 'xg_shot_avg', 'shots_avg', 'shots_on_target_percent', 'head_goals_avg',
    'assists_avg', 'xg_assist_avg', 'touch_in_box_avg', 'shot_assists_avg', 'key_passes_avg',
    'dribbles_avg', 'successful_dribbles_percent', 'progressive_run_avg', 'progressive_pass_avg',
    'successful_progressive_pass_percent', 'pre_assist_avg', 'smart_passes_avg',
    'pass_to_penalty_area_avg', 'accurate_pass_to_penalty_area_percent'
]
def calcular_scoring(df, clubes):
    subset = df[
        (df['League'].str.lower() == 'segunda rfef') &
        (df['position_gen_ESP1'].str.lower() == 'delantero') &
        (df['minutes_on_field'] >= 1500) &
        (df['last_club_name'].str.lower().isin(clubes))
    ].copy()

    subset = subset.dropna(subset=metricas_utilizadas)

    # Normalización min-max por columna
    for col in metricas_utilizadas:
        min_val = subset[col].min()
        max_val = subset[col].max()
        if min_val != max_val:  # Evita división por cero si todos los valores son iguales
            subset[f'{col}_score'] = (subset[col] - min_val) / (max_val - min_val)
        else:
            subset[f'{col}_score'] = 0  # o 1, o np.nan según tu criterio

    # Media simple de scores
    subset['impacto_ofensivo'] = subset[
        [f'{col}_score' for col in metricas_utilizadas]
    ].mean(axis=1).round(3)

    return subset.sort_values('impacto_ofensivo', ascending=False).head(25)

top_25 = calcular_scoring(df, [c.lower() for c in clubes_validos])
top_25_filial = calcular_scoring(df, [c.lower() for c in clubes_filiales])

def calcular_top25_por_grupo(df, clubes):
    df_filtrado = df[
        (df['League'].str.lower() == 'segunda rfef') &
        (df['position_gen_ESP1'].str.lower() == 'delantero') &
        (df['minutes_on_field'] >= 1500) &
        (df['last_club_name'].str.lower().isin(clubes))
    ].copy()

    df_filtrado = df_filtrado.dropna(subset=metricas_utilizadas)

    for col in metricas_utilizadas:
        min_val = df_filtrado[col].min()
        max_val = df_filtrado[col].max()
        if min_val != max_val:
            df_filtrado[f'{col}_score'] = (df_filtrado[col] - min_val) / (max_val - min_val)
        else:
            df_filtrado[f'{col}_score'] = 0

    df_filtrado['impacto_ofensivo'] = df_filtrado[
        [f'{col}_score' for col in metricas_utilizadas]
    ].mean(axis=1).round(3)

    top25_por_grupo = {}
    for grupo in sorted(df_filtrado['Group'].unique()):
        top25 = df_filtrado[df_filtrado['Group'] == grupo].sort_values('impacto_ofensivo', ascending=False).head(25)
        top25_por_grupo[grupo] = top25

    return top25_por_grupo


class PDF(FPDF):
    def __init__(self):
        super().__init__()
        self.set_auto_page_break(auto=True, margin=15)

    def portada(self, path_logo):
        self.add_page()
        self.set_font("Arial", "B", 10)
        self.cell(0, 10, "Secretaría Técnica de Unionistas de Salamanca CF", ln=True, align="C")
        self.ln(50)
        if path_logo and os.path.exists(path_logo):
            self.image(path_logo, x=(210 - 60) / 2, y=50, w=60)
            self.ln(60)
        self.set_font("Arial", "B", 18)
        self.cell(0, 10, "Informe Top 25 delanteros de 2RFEF", ln=True, align="C")
        self.ln(10)
        self.set_font("Arial", "", 14)
        self.cell(0, 10, "Separado por No Filiales y Filiales", ln=True, align="C")

    def header(self):
        if hasattr(self, 'title') and self.page_no() > 1:
            self.set_font("Arial", "B", 14)
            self.cell(0, 10, self.title, ln=True, align="C")
            self.ln(5)

    def footer(self):
        if self.page_no() == 1:
            return  # Omitir footer en portada

        self.set_y(-15)

        # Texto izquierda
        self.set_font('Arial', '', 8)
        self.set_x(10)
        self.cell(60, 10, "Informe realizado por: Ramón Codesido", align='L')

        # Número de página centrado
        self.set_x(90)  # Aproximadamente centrado para un ancho de 30-40px
        self.set_font('Arial', 'I', 8)
        self.cell(30, 10, f'Página {self.page_no() - 1}', align='C')

        # Logo derecha
        path_footer_logo = "/Users/macmontxinho/Desktop/Teams/Unionistas/Imagenes/Escudo_Of.png"
        if os.path.exists(path_footer_logo):
            self.image(path_footer_logo, x=180, y=self.get_y(), w=14)

    def add_ranking_page(self, df_ranking, title):
        self.title = title
        self.add_page()
        for i, (_, row) in enumerate(df_ranking.iterrows(), 1):
            altura_jugador = 22
            if self.get_y() + altura_jugador > 270:
                self.add_page()
            y_start = self.get_y()
            self.set_draw_color(180, 180, 180)
            self.line(10, y_start, 200, y_start)
            if pd.notna(row['image']) and row['image'].startswith('http'):
                try:
                    r = requests.get(row['image'], timeout=5)
                    if r.status_code == 200:
                        with tempfile.NamedTemporaryFile(delete=False, suffix=".png") as tmp_img:
                            tmp_img.write(r.content)
                            tmp_img.flush()
                            self.image(tmp_img.name, x=12, y=y_start + 2, w=12, h=12)
                except:
                    pass
            self.set_xy(26, y_start + 2)
            self.set_font("Arial", "B", 10)
            self.cell(0, 5, f"{i}. {row['name']}", ln=1)
            self.set_x(26)
            self.set_font("Arial", "", 9)
            self.cell(0, 5, f"Índice: {row['impacto_ofensivo']:.3f} | Minutos: {int(row['minutes_on_field'])} | Edad: {int(row['age'])} | Equipo 24/25: {row['last_club_name']}", ln=1)
            self.set_x(26)
            self.cell(0, 5, f"Equipo actual: {row['current_team_name']}", ln=1)
            self.set_y(y_start + altura_jugador - 5)
            self.line(10, self.get_y(), 200, self.get_y())
            self.ln(3)
        def add_top25_grupos_paralelo(self, dict_grupos):
            self.title = "Top 25 Delanteros por Grupo - 2RFEF"
            self.add_page()
            self.set_font("Arial", "B", 10)
            self.cell(0, 10, "Top 25 por grupo de 2ª RFEF (G1 a G5)", ln=True, align="C")
            self.ln(5)

            col_width = 38
            row_height = 5
            margin_x = 10
            max_filas = 25

            grupos = sorted(dict_grupos.keys())[:5]  # G1 a G5

            # Cabeceras
            for i, g in enumerate(grupos):
                self.set_xy(margin_x + i * col_width, self.get_y())
                self.cell(col_width, row_height, f"Grupo {g}", border=1, align='C')
            self.ln(row_height)

            # Filas de jugadores
            for fila in range(max_filas):
                for i, g in enumerate(grupos):
                    jugadores = dict_grupos[g].reset_index()
                    if fila < len(jugadores):
                        jugador = jugadores.iloc[fila]
                        texto = f"{fila+1}. {jugador['name'][:22]}"
                    else:
                        texto = ""
                    self.set_xy(margin_x + i * col_width, self.get_y())
                    self.cell(col_width, row_height, texto, border=1)
                self.ln(row_height)

        # Añadir el método a la clase PDF (si no estaba)
        PDF.add_top25_grupos_paralelo = add_top25_grupos_paralelo

    def add_metricas_page(self):
        self.title = "Métricas Seleccionadas"
        self.add_page()
        self.set_font("Arial", "B", 14)
        self.cell(0, 10, "Métricas utilizadas para el índice", ln=True, align="C")
        self.ln(5)

        def add_section(title, items):
            self.set_font("Arial", "B", 12)
            self.cell(0, 8, title, ln=True)
            self.set_font("Arial", "", 11)
            for item in items:
                self.cell(0, 8, f"- {item}", ln=True)
            self.ln(3)

        add_section("1. Finalización (mayor peso)", [
            "non_penalty_goal","non_penalty_goal_avg", "xg_shot_avg", "shots_avg", 
            "shots_on_target_percent", "head_goals_avg"
        ])
        add_section("2. Asistencia y creación directa", [
            "assists", "xg_assist_avg", "touch_in_box_avg", 
            "shot_assists_avg", "key_passes_avg"
        ])
        add_section("3. Construcción ofensiva", [
            "pre_assist_avg", "smart_passes_avg", "pass_to_penalty_area_avg", 
            "accurate_pass_to_penalty_area_percent"
        ])
        add_section("4. Progresión y desborde", [
            "dribbles_avg", "successful_dribbles_percent", 
            "progressive_run_avg", "progressive_pass_avg", 
            "successful_progressive_pass_percent"
])

    def add_explicacion_metricas(self, path_img_tabla=None):
        self.title = "Explicación del índice y su impacto"
        self.add_page()
        self.set_font("Arial", "B", 13)
        self.cell(0, 10, "¿Qué tipo de delanteros se ven favorecidos o perjudicados por este índice?", ln=True)
        self.ln(5)
        self.set_font("Arial", "", 10)
        texto = (
            "El índice favorece a:\n"
            "- Delanteros con alta producción goleadora (9 de área, rematadores)\n"
            "- Delanteros móviles con capacidad de finalizar y asistir\n"
            "- Perfiles que combinan finalización, presencia en el área y creación\n\n"
            "El índice penaliza a:\n"
            "- Delanteros con poca incidencia en el remate o el xG generado\n"
            "- Extremos o atacantes que no aportan cifras ofensivas relevantes\n\n"
            "Este índice premia la capacidad para finalizar jugadas, generar peligro y participar activamente en la fase ofensiva.\n"
            "IMPORTANTE: Se ha aplicado un filtro previo para considerar solo jugadores con mínimo 1.000 minutos disputados en 1RFEF."
        )
        self.multi_cell(0, 6, texto)
        self.ln(5)
        if path_img_tabla and os.path.exists(path_img_tabla):
            self.image(path_img_tabla, x=15, w=180)

def generar_tabla_conclusion(path_salida="tabla_conclusion.png"):
    data = {
        "Tipo de delantero": [
            "Rematador puro (9 clásico)", 
            "Delantero móvil / rupturista", 
            "Delantero versátil (remate + creación)", 
            "Segunda punta / 'enganche'", 
            "Delantero de banda reconvertido"
        ],
        "¿Favorecido?": ["Sí", "Sí", "Mucho", "Depende", "No"],
        "Motivo principal": [
            "Aporta goles, remates y presencia en el área",
            "xG, remate y desmarque, incluso con menos asociación",
            "Contribuye en remate, asistencia y juego",
            "Gran visión y asistencia, pero menos presencia en finalización",
            "Mucho desborde pero bajo impacto en xG o goles"
        ]
    }
    fig, ax = plt.subplots(figsize=(12, 3))
    ax.axis('tight')
    ax.axis('off')
    table = ax.table(cellText=list(zip(*data.values())), colLabels=list(data.keys()), cellLoc='center', loc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(9)
    table.scale(1.3, 1.6)
    for (row, col), cell in table.get_celld().items():
        if row == 0:
            cell.set_fontsize(10)
            cell.set_text_props(weight='bold')
            cell.set_facecolor('#1E2A3B')
            cell.set_text_props(color='white')
        else:
            cell.set_facecolor('#F2F2F2' if row % 2 == 0 else '#FFFFFF')
    plt.savefig(path_salida, bbox_inches='tight', dpi=300)
    plt.close()

# --------------------
# Generar informe
# --------------------
# Asegúrate de calcular esto antes de usarlo en el PDF
top25_por_grupo = calcular_top25_por_grupo(df, [c.lower() for c in clubes_validos])

ruta_logo = "/Users/macmontxinho/Desktop/Teams/Unionistas/Imagenes/IMG_2027-removebg-preview.png"
pdf = PDF()
pdf.portada(ruta_logo)
pdf.add_ranking_page(top_25, "Top 25 Delanteros - No Filiales")
pdf.add_ranking_page(top_25_filial, "Top 25 Delanteros - Filiales")
pdf.add_top25_grupos_paralelo(top25_por_grupo)
pdf.add_metricas_page()

path_tabla_img = "tabla_conclusion.png"
generar_tabla_conclusion(path_tabla_img)
pdf.add_explicacion_metricas(path_img_tabla=path_tabla_img)
pdf.output("Ranking_Delanteros_FPDF_2RFEF.pdf")

print("PDF generado correctamente.")

PDF generado correctamente.
